# Introduction/Business Problem

France is a country which tries to attract an increasing number of highly skilled migrants.
Paris-the capital city-has the status of a world-city, comparatively to New York, London or Tokyo. Paris has concentrated many multinational companies, so much so that many of the high-skilled jobs in France are solely present in Paris and nowhere else in the country (trader, financial structurer, big data engineer...).
As many studies have shown that skilled migrants have a higher propensity to create new, innovative businesses (even compared to the native population, let alone the low-skilled migrants), it could be a great investment for France to attract skilled migrants in regions outside of Paris in particular. The latter could help decrease regional disparities in the country, hoping for better public infrastructure to develop outside of the capital city.

The regions of France aren't well known to most foreigners outside of Europe. Here, as our data science project, we try to build a dashboard allowing to evaluate the different provinces of France, known as "département".
The dashboard allows to make an informed decision regarding where is best to move, based on the following data:
- real estate prices (houses and apartments)
- average net income
- number of middle and high schools
The dashboard also provides the longitude and latitude of the département, so one may easily find extra information online thanks to both metrics.

# Data 

Average net incomes:
https://www.journaldunet.com/business/salaire/classement/departements/salaires
https://www.journaldunet.com/business/salaire/classement/departements/salaires?page=2

Real estate prices:
https://www.consortium-immobilier.fr/prix/

Statistics on middle and high schools:
https://www.insee.fr/fr/statistiques/2012787#tableau-TCRD_061_tab1_departements

# Downloading addons 

In [4]:
!pip install geocoder
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


# Income data per department & longitudes-latitudes - Webscraping

In [301]:
#webscraping the income data
income=[]
url = 'https://www.journaldunet.com/business/salaire/classement/departements/salaires'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
for table in soup.find_all("table"):
    for row in table.find("tbody").find_all("tr"):
        col = row.find_all("td")
        income.append(col[0].text)
        income.append(col[1].text)
        income.append(col[2].text)
url = 'https://www.journaldunet.com/business/salaire/classement/departements/salaires?page=2'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
for table in soup.find_all("table"):
    for row in table.find("tbody").find_all("tr"):
        col = row.find_all("td")
        income.append(col[0].text)
        income.append(col[1].text)
        income.append(col[2].text)

#sorting the data by type into different lists
dpt_number=[]
dpt_name=[]
income_net=[]
i=0
while i<len(income)-2:
    dpt_name.append(income[i+1])
    income_net.append(income[i+2])
    i=i+3

#using geocoder to extract latitudes and longitudes, putting them into lists
latitude=[]
longitude=[]
i=0
while i<len(dpt_name):
    address = geocoder.arcgis(dpt_name[i])
    lat_long_coordinates=address.latlng
    latitude_data=lat_long_coordinates[0]
    longitude_data=lat_long_coordinates[1]
    latitude.append(latitude_data)
    longitude.append(longitude_data)
    i=i+1

#changing strings into integers so we can rank departments according to average income levels
income_cleared=[]
i = 0
while i<len(income_net):
    income_cleared.append(income_net[i][0:5])
    i=i+1
i = 0
income_last_part=[]
while i<len(income_cleared):
    income_last_part.append(income_cleared[i][-3:len(income_cleared)+1])
    i=i+1
income_first_part=[]
i=0
while i<len(income_cleared):
    income_first_part.append(income_cleared[i][0:1])
    income_first_part[i]=int(income_first_part[i]+income_last_part[i])
    i=i+1
income_net=income_first_part

#putting the data into a dataframe
income_tab = pd.DataFrame({'Department' : dpt_name,
                            'Net income (monthly, EUR)' : income_net, 
                            'Longitude' : longitude,
                            'Latitude' : latitude}, 
                            columns=['Department', 'Net income (monthly, EUR)', 'Longitude', 'Latitude'])

# Housing data per department - Webscraping

In [286]:
#webscraping housing data
housing=[]
url = 'https://www.consortium-immobilier.fr/prix/'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
for table in soup.find_all("table"):
    for row in table.find("tbody").find_all("tr"):
        col = row.find_all("td")
        housing.append(col[0].text)
        housing.append(col[1].text)
        housing.append(col[2].text)

#clearing the data, separating useless values
i=0
while i<len(housing):
    if housing[i] == '01 - Ain':
        saver_1=i
    if housing[i] == '1530 €':
        saver_2=i+1
    i=i+1
housing=housing[saver_1:saver_2]

#classifying the data by type (house prices, apartment prices, department names)
house_price=[]
apt_price=[]
dpt_price=[]
i=0
while i<len(housing)-2:
    dpt_price.append(housing[i])
    apt_price.append(housing[i+1])
    house_price.append(housing[i+2])
    i=i+3

#turning strings into integer so we can order departments based on house and apartment prices
i = 0
while i<len(house_price):
    house_price[i]=int(house_price[i][0:4])
    i=i+1
i=0
while i<len(apt_price):
    apt_price[i]=int(apt_price[i][0:4])
    i=i+1

# School data per department - Webscraping

In [287]:
#webscraping school data
school=[]
url = 'https://www.insee.fr/fr/statistiques/2012787#tableau-TCRD_061_tab1_departements'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
for table in soup.find_all("table"):
    for row in table.find("tbody").find_all("tr"):
        col = row.find_all("td")
        col_0 = row.find_all("th")
        school.append(col_0[0].text)
        school.append(col[0].text)
        school.append(col[1].text)
        school.append(col[2].text)
        school.append(col[3].text)
        school.append(col[4].text)
        school.append(col[5].text)
        school.append(col[6].text)
        school.append(col[7].text)

i=0
while i<len(school):
    if school[i] == "Val-d'Oise":
        saver_1=i
    i=i+1

school=school[0:saver_1+9]

#ordering the webscrapped data by type into different lists
col_dpt=[]
col_0=[]
col_1=[]
col_2=[]
col_4=[]
col_5=[]
col_6=[]
col_7=[]
i=0
while i<len(school)-8:
    col_dpt.append(school[i])
    col_0.append(school[i+1])
    col_1.append(school[i+2])
    col_2.append(school[i+3])
    col_4.append(school[i+5])
    col_5.append(school[i+6])
    col_6.append(school[i+7])
    col_7.append(school[i+8])
    i=i+9

#making Corse-du-Sud and Haute-Corse only one department named Corse
i=0
while i<len(col_dpt)-1:
    if col_dpt[i]=="Corse-du-Sud":
        col_dpt[i]="Corse"
        col_0[i]=int(col_0[i])+int(col_0[i+1])
        col_1[i]=int(col_1[i])+int(col_1[i+1])
        col_2[i]=int(col_2[i])+int(col_2[i+1])
        col_4[i]=int(col_4[i])+int(col_2[i+1])
        col_5[i]=int(col_5[i])+int(col_5[i+1])
        col_6[i]=int(col_6[i])+int(col_6[i+1])
        col_7[i]=int(col_7[i])+int(col_7[i+1])  
        i=i+1
        while i<len(col_dpt)-1:
            col_dpt[i]=col_dpt[i+1]
            col_0[i]=col_0[i+1]
            col_1[i]=col_1[i+1]
            col_2[i]=col_2[i+1]
            col_4[i]=col_4[i+1]
            col_5[i]=col_5[i+1]
            col_6[i]=col_6[i+1]
            col_7[i]=col_7[i+1]
            i=i+1
    i=i+1

#converting string data into integer to be able to rank departments based on school data
i=0
while i<len(col_dpt)-1:
        col_0[i]=int(col_0[i])
        col_1[i]=int(col_1[i])
        col_2[i]=int(col_2[i])
        col_4[i]=int(col_4[i])
        col_5[i]=int(col_5[i])
        col_6[i]=int(col_6[i])
        i=i+1

#deleting the last row
k=len(col_dpt)-1
col_dpt=col_dpt[0:k]
col_0=col_0[0:k]
col_1=col_1[0:k]
col_2=col_2[0:k]
col_4=col_4[0:k]
col_5=col_5[0:k]
col_6=col_6[0:k]
col_7=col_7[0:k]

#setting the dataframe
schools = pd.DataFrame({'Department' : col_dpt,
                                'Public middle school' : col_0,
                                'Public professional high school' : col_1,
                                'Public general high school' : col_2,
                                'Private middle school' : col_4,
                                'Private professional high school' : col_5,
                                'Private general high school' : col_6,
                                'House price (EUR/m2)' : house_price,
                                'Apartment price (EUR/m2)' : apt_price}, 
                                columns=['Department','Public middle school','Public professional high school', 'Public general high school','Private middle school','Private professional high school','Private general high school','House price (EUR/m2)','Apartment price (EUR/m2)'])

# Departments with highest incomes

In [288]:
#ordering the departments based on (see title), extracting top 5
a=income_tab.sort_values(by=['Net income (monthly, EUR)'], ascending = False).head(5)
print(a['Department'])
a_1=[]
a_1=a['Department']
a_2 =[a_1[0], a_1[1], a_1[2], a_1[3], a_1[4]]

0             Paris
1    Hauts-de-Seine
2          Yvelines
3      Val-de-Marne
4           Essonne
Name: Department, dtype: object


# Departments with most public middle schools

In [289]:
#ordering the departments based on (see title), extracting top 5
b=schools.sort_values(by=['Public middle school'], ascending = False).head(5)
print(b['Department'])
b_1=[]
b_1=b['Department']
b_2 =[b_1[58], b_1[12], b_1[92], b_1[76], b_1[61]]

58                 Nord
12     Bouches-du-Rhône
92    Seine-Saint-Denis
76       Seine-et-Marne
61        Pas-de-Calais
Name: Department, dtype: object


# Departments with most public professional high schools

In [290]:
#ordering the departments based on (see title), extracting top 5
c=schools.sort_values(by=['Public professional high school'], ascending = False).head(5)
print(c['Department'])
c_1=[]
c_1=c['Department']
c_2 =[c_1[58], c_1[74], c_1[61], c_1[12], c_1[68]]

58                Nord
74               Paris
61       Pas-de-Calais
12    Bouches-du-Rhône
68               Rhône
Name: Department, dtype: object


# Departments with most public general high schools

In [291]:
#ordering the departments based on (see title), extracting top 5
d=schools.sort_values(by=['Public general high school'], ascending = False).head(5)
print(d['Department'])
d_1=[]
d_1=d['Department']
d_2 =[d_1[74], d_1[58], d_1[92], d_1[76], d_1[77]]

74                Paris
58                 Nord
92    Seine-Saint-Denis
76       Seine-et-Marne
77             Yvelines
Name: Department, dtype: object


# Departments with most private middle schools

In [292]:
#ordering the departments based on (see title), extracting top 5
e=schools.sort_values(by=['Private middle school'], ascending = False).head(5)
print(e['Department'])
e_1=[]
e_1=e['Department']
e_2 =[e_1[58], e_1[74], e_1[68], e_1[43], e_1[12]]

58                Nord
74               Paris
68               Rhône
43    Loire-Atlantique
12    Bouches-du-Rhône
Name: Department, dtype: object


# Departments with most private professional high schools

In [293]:
#ordering the departments based on (see title), extracting top 5
f=schools.sort_values(by=['Private professional high school'], ascending = False).head(5)
print(f['Department'])
f_1=[]
f_1=f['Department']
f_2 =[f_1[68], f_1[12], f_1[41], f_1[32], f_1[30]]

68               Rhône
12    Bouches-du-Rhône
41               Loire
32             Gironde
30       Haute-Garonne
Name: Department, dtype: object


# Departments with most private general high schools

In [294]:
#ordering the departments based on (see title), extracting top 5
g=schools.sort_values(by=['Private general high school'], ascending = False).head(5)
print(g['Department'])
g_1=[]
g_1=g['Department']
g_2 =[g_1[74], g_1[12], g_1[58], g_1[68], g_1[43]]

74               Paris
12    Bouches-du-Rhône
58                Nord
68               Rhône
43    Loire-Atlantique
Name: Department, dtype: object


# Departments with most expensive houses

In [295]:
#ordering the departments based on (see title), extracting top 5
h=schools.sort_values(by=['House price (EUR/m2)'], ascending = True).head(5)
print(h['Department'])
h_1=[]
h_1=h['Department']
h_2 =[h_1[72], h_1[43], h_1[30], h_1[15], h_1[94]]

72              Savoie
43    Loire-Atlantique
30       Haute-Garonne
15            Charente
94          Val-d'Oise
Name: Department, dtype: object


# Departments with most expensive apartments

In [296]:
#ordering the departments based on (see title), extracting top 5
i=schools.sort_values(by=['Apartment price (EUR/m2)'], ascending = True).head(5)
print(i['Department'])
i_1=[]
i_1=i['Department']
i_2 =[i_1[68], i_1[66], i_1[11], i_1[51], i_1[39]]

68          Rhône
66       Bas-Rhin
11        Aveyron
51    Haute-Marne
39         Landes
Name: Department, dtype: object


# Summary tables

In [297]:
#setting the dataframe to form a summary table
results = pd.DataFrame({'Dpt with highest incomes' : a_2,
                                'Dpt with most public middle schools' : b_2,
                                'Dpt with most public professional high schools' : c_2,
                                'Dpt with most public general high schools' : d_2,
                                'Dpt with most private middle schools' : e_2,
                                'Dpt with most private professional high schools' : f_2,
                                'Dpt with most private general high schools' : g_2,
                                'Dpt with most expensive houses' : h_2,
                                'Dpt with most expensive apartments' : i_2}, 
                                columns=['Dpt with highest incomes','Dpt with most public middle schools','Dpt with most public professional high schools', 'Dpt with most public general high schools','Dpt with most private middle schools','Dpt with most private professional high schools','Dpt with most private general high schools','Dpt with most expensive houses','Dpt with most expensive apartments'])
results

,Dpt with highest incomes,Dpt with most public middle schools,Dpt with most public professional high schools,Dpt with most public general high schools,Dpt with most private middle schools,Dpt with most private professional high schools,Dpt with most private general high schools,Dpt with most expensive houses,Dpt with most expensive apartments
0,Paris,Nord,Nord,Paris,Nord,Rhône,Paris,Savoie,Rhône
1,Hauts-de-Seine,Bouches-du-Rhône,Paris,Nord,Paris,Bouches-du-Rhône,Bouches-du-Rhône,Loire-Atlantique,Bas-Rhin
2,Yvelines,Seine-Saint-Denis,Pas-de-Calais,Seine-Saint-Denis,Rhône,Loire,Nord,Haute-Garonne,Aveyron
3,Val-de-Marne,Seine-et-Marne,Bouches-du-Rhône,Seine-et-Marne,Loire-Atlantique,Gironde,Rhône,Charente,Haute-Marne
4,Essonne,Pas-de-Calais,Rhône,Yvelines,Bouches-du-Rhône,Haute-Garonne,Loire-Atlantique,Val-d'Oise,Landes


In [304]:
income_tab.sort_values(by=['Net income (monthly, EUR)'], ascending = False).head(30)

,Department,"Net income (monthly, EUR)",Longitude,Latitude
0,Paris,3683,2.341400,48.857170
1,Hauts-de-Seine,3585,2.241225,48.806163
2,Yvelines,3268,1.841223,48.815273
3,Val-de-Marne,2666,2.469195,48.777422
4,Essonne,2601,2.243431,48.522547
5,Rhône,2487,4.640873,45.871030
6,Val-d'Oise,2467,2.130822,49.082732
7,Haute-Garonne,2466,1.174886,43.358858
8,Seine-et-Marne,2430,2.934009,48.627462
9,Isère,2373,5.574199,45.263864


In [305]:
schools.sort_values(by=['Private general high school'], ascending = False).head(30)

,Department,Public middle school,Public professional high school,Public general high school,Private middle school,Private professional high school,Private general high school,House price (EUR/m2),Apartment price (EUR/m2)
74,Paris,114,29,71,78,12,103,1767,2043
12,Bouches-du-Rhône,136,26,39,65,27,53,2425,2107
58,Nord,203,44,56,84,16,48,1743,2499
68,Rhône,113,24,37,72,31,43,1874,1500
43,Loire-Atlantique,83,15,25,68,14,29,1518,2129
77,Yvelines,116,7,42,37,9,27,1658,2489
91,Hauts-de-Seine,99,13,40,42,6,26,2192,1606
5,Alpes-Maritimes,73,13,22,21,9,24,1792,2224
93,Val-de-Marne,105,10,37,38,9,23,2206,2307
92,Seine-Saint-Denis,130,16,51,35,8,22,1677,1869
